# swisslandstats preprocessing

We will create an extract of the Canton of Vaud (Switerland) derived from the [Swiss Land Statistics (SLS) datasets from the Swiss Federal Statistical Office](https://www.bfs.admin.ch/bfs/en/home/services/geostat/swiss-federal-statistics-geodata/land-use-cover-suitability/swiss-land-use-statistics.html). We will first use GNU Make to download the data:

In [ ]:
!make -C .. download_lulc

This downloads the SLS inventory as a CSV file:

In [ ]:
!ls ../data/raw/lulc

Each row corresponds to one of the hectometric pixels that configure the Swiss territory, and features three main goups of columns [1], i.e.:

* the coordinates of the pixels centroid `E`, `N` in the LV95 coordinate reference system (CRS) or `X`, `Y` in the LV03 CRS
* columns starting with `FJ` denote the exact year when the observation was taken
* columns starting with `AS` denote the actual land use/land cover (LULC) code of each pixel

This is what the file looks like:

In [ ]:
!head ../data/raw/lulc/AREA_NOAS04_17_181029.csv

We will now preprocess this data using the [swisslandstats-geopy](https://github.com/martibosch/swisslandstats-geopy) library:

In [ ]:
import swisslandstats as sls

Below are the parameters needed to run this notebook (to be filled directly in the notebook or using [papermill](https://github.com/nteract/papermill)):

In [ ]:
lulc_csv_filepath = ""
lulc_column = ""
dst_filepath = ""
nominatim_query = "District de Veveyse, Switzerland"

We will now:

1. read the data into a land data frame

In [ ]:
ldf = sls.read_csv(lulc_csv_filepath)

2. clip it to the extent of interest:

In [ ]:
extent_ldf = ldf.clip_by_nominatim(nominatim_query)

3. dump the LULC columns of interest into a geotiff file:

In [ ]:
extent_ldf.to_geotiff(dst_filepath, lulc_column)

## References

1. Bosch, M. (2019). swisslandstats-geopy: Python tools for the land statistics datasets from the Swiss Federal Statistical Office. Journal of Open Source Software, 4(41), 1511.